### Import a market place model into bedrock

Use Amazon Bedrock Marketplace to discover, test, and use over 100 popular, emerging, and specialized foundation models (FMs). These models are in addition to the selection of industry-leading models in Amazon Bedrock.

You can discover the models in a single catalog. After you discover the model, you can subscribe to it and deploy it to an endpoint managed by SageMaker AI.

You can access the models that you’ve deployed through Amazon Bedrock’s APIs. Accessing the models through Amazon Bedrock’s APIs allows you to use them natively with Amazon Bedrock's tools such as Agents, Knowledge Bases, and Guardrails.

You can access the Amazon Bedrock Marketplace models from the: InvokeModel operation, Converse operation, Amazon Bedrock console

https://docs.aws.amazon.com/bedrock/latest/userguide/amazon-bedrock-marketplace.html

In [65]:
import time
import json
import boto3
from botocore.config import Config

In [46]:
# Initialize Bedrock Runtime client
session = boto3.Session()
client = session.client(
    service_name='bedrock-runtime',
    config=Config(
        connect_timeout=300,  # 5 minutes
        read_timeout=300,     # 5 minutes
        retries={'max_attempts': 1}
    )
)

In [47]:
model_id = "arn:aws:sagemaker:us-west-2:7076*********:endpoint/endpoint-quick-start-j0s02"

In [58]:
def chat_completion(prompt, temperature=0.6, max_tokens=1024, top_p=0.9, model_id=model_id, max_retries=3):
    """
    Simplified completion with retry mechanism using Bedrock invoke_model
    
    Parameters:
        prompt (str): The input prompt
        temperature (float): Controls randomness (0.0-1.0)
        max_tokens (int): Max tokens to generate
        top_p (float): Nucleus sampling parameter
        model_id (str): Model identifier
        max_retries (int): Max retry attempts
    
    Returns:
        dict: Model response with generated text
    """
    
    request_body = {
        "inputs": prompt,
        "parameters": {
            "max_new_tokens": max_tokens,
            "stop": [],
            "temperature": temperature,
            "top_p": top_p,
            "top_k": None
        }
    }

    for attempt in range(max_retries):
        try:
            response = client.invoke_model_with_response_stream(
                body=json.dumps(request_body).encode('utf-8'),
                modelId=model_id,
                contentType='application/json',
                accept='application/json',
                performanceConfigLatency='standard'
            )
            
            generated_text = None
            for event in response['body']:
                if 'chunk' in event:
                    chunk_data = json.loads(event['chunk']['bytes'].decode())
                    if chunk_data.get('generated_text'):
                        generated_text = chunk_data['generated_text']
                        
            
            return {"generated_text": generated_text if generated_text else ""}
            
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {str(e)}")
            if attempt < max_retries - 1:
                time.sleep(30)
            continue

    raise Exception("Max retries reached")

In [59]:
# Test usage
prompt = """Alice and Bob play the following game. A stack of $n$ tokens lies before them. 
The players take turns with Alice going first. On each turn, the player removes either $1$ token 
or $4$ tokens from the stack. Whoever removes the last token wins. Find the number of positive 
integers $n$ less than or equal to $2024$ for which there exists a strategy for Bob that 
guarantees that Bob will win the game regardless of Alice's play.  """

In [56]:
print("=== Test Case ===")
response = chat_completion(prompt)
print(response)

=== Test Case ===
{'generated_text': "Alice and Bob play the following game. A stack of $n$ tokens lies before them. \nThe players take turns with Alice going first. On each turn, the player removes either $1$ token \nor $4$ tokens from the stack. Whoever removes the last token wins. Find the number of positive \nintegers $n$ less than or equal to $2024$ for which there exists a strategy for Bob that \nguarantees that Bob will win the game regardless of Alice's play.  \n\n\nAlright, so Alice and Bob are playing this game where they take turns removing either 1 or 4 tokens from a stack of n tokens. Alice goes first, and the person who takes the last token wins. We need to find how many positive integers n ≤ 2024 where Bob can guarantee a win, no matter how Alice plays.\n\nHmm, okay. So, first, let me try to understand the game. It's similar to a Nim game but with specific move options: each player can remove 1 or 4 tokens on their turn. The goal is to take the last token.\n\nI think the

### Testing the throughput and lantency with locust

In [62]:
!pip install locust

In [70]:
%%writefile locustfile.py

from locust import User, task, between
import logging

import boto3
import json
from botocore.config import Config

# Initialize Bedrock Runtime client
session = boto3.Session()
client = session.client(
    service_name='bedrock-runtime',
    config=Config(
        connect_timeout=600,  # 10 minutes
        read_timeout=600,     # 10 minutes
        retries={'max_attempts': 3}
    )
)

model_id = "arn:aws:sagemaker:us-west-2:7076********:endpoint/endpoint-quick-start-j0s02"


def chat_completion(prompt, temperature=0.6, max_tokens=1024, top_p=0.9, model_id=model_id, max_retries=3):
    """
    Simplified completion with retry mechanism using Bedrock invoke_model
    
    Parameters:
        prompt (str): The input prompt
        temperature (float): Controls randomness (0.0-1.0)
        max_tokens (int): Max tokens to generate
        top_p (float): Nucleus sampling parameter
        model_id (str): Model identifier
        max_retries (int): Max retry attempts
    
    Returns:
        dict: Model response with generated text
    """
    
    request_body = {
        "inputs": prompt,
        "parameters": {
            "max_new_tokens": max_tokens,
            "stop": [],
            "temperature": temperature,
            "top_p": top_p,
            "top_k": None
        }
    }

    for attempt in range(max_retries):
        try:
            response = client.invoke_model_with_response_stream(
                body=json.dumps(request_body).encode('utf-8'),
                modelId=model_id,
                contentType='application/json',
                accept='application/json',
                performanceConfigLatency='standard'
            )
            
            generated_text = None
            for event in response['body']:
                if 'chunk' in event:
                    chunk_data = json.loads(event['chunk']['bytes'].decode())
                    if chunk_data.get('generated_text'):
                        generated_text = chunk_data['generated_text']
                        
            
            return {"generated_text": generated_text if generated_text else ""}
            
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {str(e)}")
            if attempt < max_retries - 1:
                time.sleep(30)
            continue

    raise Exception("Max retries reached")

# Test usage
prompt = """Alice and Bob play the following game. A stack of $n$ tokens lies before them. 
The players take turns with Alice going first. On each turn, the player removes either $1$ token 
or $4$ tokens from the stack. Whoever removes the last token wins. Find the number of positive 
integers $n$ less than or equal to $2024$ for which there exists a strategy for Bob that 
guarantees that Bob will win the game regardless of Alice's play."""

    
class LLMUser(User):
    @task
    def generation(self):
        # Invoke the model
        with self.environment.events.request.measure("[Send]", "Prompt"):
            response = chat_completion(prompt)
            logging.info(response.get('generated_text')[:200])
            
        logging.info("Finished generation!")            

Overwriting locustfile.py


The configuration with Command Line Options https://docs.locust.io/en/stable/configuration.html

--users Peak number of concurrent Locust users. Primarily used together with --headless or --autostart.

--headless Disable the web interface, and start the test immediately.

--csv Store request stats to files in CSV format.

--spawn-rate Rate to spawn users at (users per second)

In this example, the --users option sets the total number of users to 30, and the --spawn-rate option sets the rate of user spawning to 30 users per second. By using the same value for --spawn-rate as the total number of users, all 30 users will be spawned immediately. Therefore, at any given time during the test, there will be a maximum of 30 concurrent users.

Please note that the --run-time option sets the duration of the test in seconds. In this example, the test will run for 120 seconds before stopping.

!locust --headless --users 30 --spawn-rate 30 --run-time 120 --csv ./benchmark_metric/benchmark_u30



In [ ]:
!locust --headless --users 30 --spawn-rate 30 --run-time 1200 --csv ./benchmark_metric/benchmark_u30

[2025-02-12 07:37:03,945] ip-172-16-23-227/INFO/locust.main: Starting Locust 2.32.8
[2025-02-12 07:37:03,945] ip-172-16-23-227/INFO/locust.main: Run time limit set to 1200 seconds
Type     Name  # reqs      # fails |    Avg     Min     Max    Med |   req/s  failures/s
--------||-------|-------------|-------|-------|-------|-------|--------|-----------
--------||-------|-------------|-------|-------|-------|-------|--------|-----------
         Aggregated       0     0(0.00%) |      0       0       0      0 |    0.00        0.00

[2025-02-12 07:37:03,946] ip-172-16-23-227/INFO/locust.runners: Ramping to 30 users at a rate of 30.00 per second
[2025-02-12 07:37:03,948] ip-172-16-23-227/INFO/locust.runners: All users spawned: {"LLMUser": 30} (30 total users)
Type     Name  # reqs      # fails |    Avg     Min     Max    Med |   req/s  failures/s
--------||-------|-------------|-------|-------|-------|-------|--------|-----------
--------||-------|-------------|-------|-------|-------|-----